In [ ]:
import pandas as pd
import polars as pl
import numpy as np
import networkx as nx
import sys
sys.path.insert(1, '../framework')
from racetrack import *
rt = RACETrack()

In [ ]:
df = pl.DataFrame({'fm': ['a','b','c', None, None, None], 
                   'to': ['b','c','a', None, None, None],
                   'src':[None, None, None, 'x', 'y', 'z'],
                   'dst':[None, None, None, 'y', 'z', 'x']})
pos = {'a':(0,0), 'b':(1,0), 'c':(1,1), 'x':(10,10), 'y':(11,10), 'z':(11,11)}
_params_ = {'df':df, 'relationships':[('fm','to'),('src','dst')], 'pos':pos}
_link_, _linknode_ = rt.link(**_params_) , rt.linkNode(**_params_)
_params_['node_size']='nil'
_link_nil_, _linknode_nil_ = rt.link(**_params_) , rt.linkNode(**_params_)
_params_['node_size']='small'
_link_sm_, _linknode_sm_ = rt.link(**_params_) , rt.linkNode(**_params_)
_params_['node_size']='medium'
_link_med_, _linknode_med_ = rt.link(**_params_) , rt.linkNode(**_params_)
_params_['node_size']='large'
_link_lrg_, _linknode_lrg_ = rt.link(**_params_) , rt.linkNode(**_params_)
rt.table([_link_, _linknode_, 
         _link_nil_, _linknode_nil_,
         _link_sm_,  _linknode_sm_,
         _link_med_, _linknode_med_,
         _link_lrg_, _linknode_lrg_
         ], per_row=4)

In [ ]:
df = pl.DataFrame({'fm': ['a','b','c'], 
                   'to': ['b','c','a']})
pos = {'a':(0,3), 'b':(5,3), 'c':(5,1), 'x':(10,10), 'y':(11,10), 'z':(11,11)}
_link2_     = rt.link    (df, [('fm','to')], pos)
_link2node_ = rt.linkNode(df, [('fm','to')], pos)
rt.tile([_link2_, _link2node_])

In [ ]:
df = pl.DataFrame({'fm': ['a','b','c'], 
                   'to': ['b','c','a']})
pos = {'a':(0,3), 'b':(5,3), 'c':(5,1)}
_link3_     = rt.link    (df, [('fm','to')], pos)
_link3node_ = rt.linkNode(df, [('fm','to')], pos)
rt.tile([_link3_, _link3node_])

In [ ]:
df = pl.DataFrame({'fm': 'a b c a b a b a b a b'.split(), 
                   'to': 'b c a b a b a b a b a'.split()})
pos = {'a':(0,3), 'b':(5,3), 'c':(5,1)}
_params4_  = {'df':df, 'relationships':[('fm','to')], 'pos':pos, 'node_size':'vary'}
_link4_     = rt.link    (**_params4_)
_link4node_ = rt.linkNode(**_params4_)
rt.tile([_link4_, _link4node_])

In [ ]:
import random
import time

_fms_, _tos_ = [], []
for i in range(10_000):
    _fms_.append(random.randint(1,10_000))
    _tos_.append(random.randint(1,10_000))
df = pl.DataFrame({'fm':_fms_, 'to':_tos_})
pos = {}
for x in set(df['fm'])|set(df['to']): pos[x] = (random.random(), random.random())
parms = {'df':df, 'relationships':[('fm','to')], 'pos':pos, 'node_size':'vary'}
t0 = time.time()
_link_ = rt.link    (**parms)
_link_._repr_svg_()
t1 = time.time()
_linknode_ = rt.linkNode(**parms)
_linknode_._repr_svg_()
t2 = time.time()

#
# link    |  linkNode    | date             | rows    | cpu/mem      |     | comments
# 2.7s    |  60.7s       | 2024-06-10 21:30 | 1M      | M1 Pro / 16g |     |
# 2.0s    |  23.6s       | 2024-06-11 19:33 | 1M      | 7900x  / 96g |     |
# 0.3s    |   2.7s       | 2024-06-11 19:35 | 100K    | 7900x  / 96g |     |
# 1.9s    |  23.6s       | 2024-06-11 20:19 | 1M      | 7900x  / 96g |     | node groupby implemented
# 0.4s    |   5.5s       | 2024-06-12 08:30 | 100K    | M1 Pro / 16g | 20K | (baseline for today)
# 0.4s    |   5.5s       | 2024-06-12 08:35 | 100K    | M1 Pro / 16g | 20K | basic node sizes done
# 0.4s    |   6.1s       | 2024-06-12 08:50 | 100K    | M1 Pro / 16g | 20K | vary node size parameter
# 0.7s    |  10.2s       | 2024-06-12 08:55 | 200K    | M1 Pro / 16g | 20K | <same ... double the rows>
#
print(f'link = {t1-t0:0.1f} seconds | linkNode = {t2-t1:0.1f} seconds | nodes = {len(pos)}')